In [100]:
import psycopg2
import pandas as pd
from datetime import datetime
import pytz
from rapidfuzz import fuzz, process
from datetime import datetime, time
import uuid

In [101]:
df = pd.read_csv('databaseUpdated2024-12-07 (1).csv')
df.head()

,fight,redCorner,blueCorner,winner,event,referee,method_of_victory,date,venue,title_fight,...,redCorner_subs_attempted,blueCorner_subs_attempted,round,time,redCorner_height,blueCorner_height,redCorner_reach,blueCorner_reach,redCorner_stance,blueCorner_stance
0,Petr Yan vs Deiveson Figueiredo,Petr Yan,Deiveson Figueiredo,Petr Yan,UFC Fight Night: Yan vs. Figueiredo,Mike Beltran,Decision - Unanimous,11.23.2024,Galaxy Arena,no,...,0,0,5,5:00,"5' 7""","5' 5""","67""","68""",Switch,Orthodox
1,Xiaonan Yan vs Tabatha Ricci,Yan Xiaonan,Tabatha Ricci,Yan Xiaonan,UFC Fight Night: Yan vs. Figueiredo,Marc Goddard,Decision - Unanimous,11.23.2024,Galaxy Arena,no,...,0,0,3,5:00,"5' 5""","5' 1""","63""","61""",Orthodox,Orthodox
2,Kenan Song vs Muslim Salikhov,Song Kenan,Muslim Salikhov,Muslim Salikhov,UFC Fight Night: Yan vs. Figueiredo,Mark Craig,KO/TKO,11.23.2024,Galaxy Arena,no,...,0,0,1,3:49,"6' 0""","5' 11""","71""","70""",Orthodox,Orthodox
3,Cong Wang vs Gabriella Fernandes,Wang Cong,Gabriella Fernandes,Gabriella Fernandes,UFC Fight Night: Yan vs. Figueiredo,Marc Goddard,Submission,11.23.2024,Galaxy Arena,no,...,0,1,2,3:49,"5' 6""","5' 6""","66""","66""",Southpaw,Southpaw
4,Volkan Oezdemir vs Carlos Ulberg,Volkan Oezdemir,Carlos Ulberg,Carlos Ulberg,UFC Fight Night: Yan vs. Figueiredo,Mike Beltran,Decision - Unanimous,11.23.2024,Galaxy Arena,no,...,0,0,3,5:00,"6' 2""","6' 4""","75""","77""",Orthodox,Orthodox


In [102]:
#Dropping all rows where red and blue corners are not recorded correctly
#Not recorded correctly until 03.21.2010
#Since this will affect the accuracy of the model I am removing it for now
#Will try and find a credible source to fix this at a later date

rows, columns = df.shape

event_condition = df['event'] == 'UFC 110: Nogueira vs Velasquez'
fight_condition = df['fight'] == 'Cain Velasquez vs Antonio Rodrigo Nogueira'

recordedIncorrectly = df.loc[event_condition & fight_condition].index[0]
recordedIncorrectly -= rows
print(recordedIncorrectly)
df = df.iloc[:recordedIncorrectly]

-1306


In [103]:
df_flipped = df.iloc[::-1].reset_index(drop=True)
df_flipped.head()

,fight,redCorner,blueCorner,winner,event,referee,method_of_victory,date,venue,title_fight,...,redCorner_subs_attempted,blueCorner_subs_attempted,round,time,redCorner_height,blueCorner_height,redCorner_reach,blueCorner_reach,redCorner_stance,blueCorner_stance
0,Jason Brilz vs Eric Schafer,Eric Schafer,Jason Brilz,Jason Brilz,UFC Live: Vera vs Jones,Tom Johnson,Decision - Unanimous,03.21.2010,The Odeum Colorado,no,...,1,1,3,5:00,"6' 3""","5' 11""","76""","71""",Orthodox,Orthodox
1,Mike Pierce vs Julio Paulino,Mike Pierce,Julio Paulino,Mike Pierce,UFC Live: Vera vs Jones,Tim Mills,Decision - Unanimous,03.21.2010,The Odeum Colorado,no,...,0,0,3,5:00,"5' 8""","6' 0""","71""",73,Orthodox,Orthodox
2,Brendan Schaub vs Chase Gormley,Brendan Schaub,Chase Gormley,Brendan Schaub,UFC Live: Vera vs Jones,Adam Martinez,KO/TKO,03.21.2010,The Odeum Colorado,no,...,0,0,1,0:47,"6' 4""","6' 3""","78""",77,Orthodox,Orthodox
3,John Howard vs Daniel Roberts,John Howard,Daniel Roberts,John Howard,UFC Live: Vera vs Jones,Tom Johnson,KO/TKO,03.21.2010,The Odeum Colorado,no,...,0,1,1,2:01,"5' 7""","5' 10""","72""","74""",Orthodox,Southpaw
4,Darren Elkins vs Duane Ludwig,Duane Ludwig,Darren Elkins,Darren Elkins,UFC Live: Vera vs Jones,Tim Mills,KO/TKO,03.21.2010,The Odeum Colorado,no,...,0,0,1,0:44,"5' 10""","5' 10""","70""","71""",Orthodox,Orthodox


In [104]:

side = {}
for index, row in df_flipped.iterrows():
    redFlag = False
    blueFlag = False
    event = row['event']
    date = row['date']
    redCorner = [row['redCorner'], row['redCorner_wins'], row['redCorner_losses'], row['redCorner_draws']]
    blueCorner = [row['blueCorner'], row['blueCorner_wins'], row['blueCorner_losses'], row['blueCorner_draws']]
    redCornerFuzzy = redCorner[0].lower().replace(" ", "").replace("-", "").replace("-", "").replace('"', '').replace("'", "")
    blueCornerFuzzy = blueCorner[0].lower().replace(" ", "").replace("-", "").replace("-", "").replace('"', '').replace("'", "")
    for key, values in side.items():
        if redFlag == True:
            break
        for value in values:
            if redCornerFuzzy == str(value).lower().replace(" ", "").replace("-", "").replace("-", "").replace('"', '').replace("'", ""):
                if int(redCorner[1])-1 == int(side[key][1]) and int(redCorner[2]) == int(side[key][2]) and int(redCorner[3]) == int(redCorner[3]):
                    #side confirmed - update record
                    side[key][1] = redCorner[1]
                    side[key][2] = redCorner[2]
                    side[key][3] = redCorner[3]
                    redFlag = True
                    break
                elif int(redCorner[1]) == int(side[key][1]) and int(redCorner[2])-1 == int(side[key][2]) and int(redCorner[3]) == int(redCorner[3]):
                    #side confirmed - update record
                    side[key][1] = redCorner[1]
                    side[key][2] = redCorner[2]
                    side[key][3] = redCorner[3]
                    redFlag = True
                    break
                elif int(redCorner[1]) == int(side[key][1]) and int(redCorner[2]) == int(side[key][2]) and int(redCorner[3])-1 == int(redCorner[3]):
                    #side confirmed - update record
                    side[key][1] = redCorner[1]
                    side[key][2] = redCorner[2]
                    side[key][3] = redCorner[3]
                    redFlag = True
                    break
            if fuzz.ratio(redCornerFuzzy, str(value).lower().replace(" ", "").replace("-", "").replace("-", "").replace('"', '').replace("'", "")) >=90:
                if int(redCorner[1])-1 == int(side[key][1]) and int(redCorner[2]) == int(side[key][2]) and int(redCorner[3]) == int(redCorner[3]):
                    #side confirmed - add alias
                    side[key].append(redCorner[0])
                    redFlag = True
                    break
                elif int(redCorner[1]) == int(side[key][1]) and int(redCorner[2])-1 == int(side[key][2]) and int(redCorner[3]) == int(redCorner[3]):
                    #side confirmed - add alias
                    side[key].append(redCorner[0])
                    redFlag = True
                    break
                elif int(redCorner[1]) == int(side[key][1]) and int(redCorner[2]) == int(side[key][2]) and int(redCorner[3])-1 == int(redCorner[3]):
                    #side confirmed - add alias
                    side[key].append(redCorner[0])
                    redFlag = True
                    break


    for key, values in side.items():
        if blueFlag == True:
            break
        for value in values:
            if blueCornerFuzzy == str(value).lower().replace(" ", "").replace("-", "").replace("-", "").replace('"', '').replace("'", ""):
                if int(blueCorner[1])-1 == int(side[key][1]) and int(blueCorner[2]) == int(side[key][2]) and int(blueCorner[3]) == int(blueCorner[3]):
                    #side confirmed - update record
                    side[key][1] = blueCorner[1]
                    side[key][2] = blueCorner[2]
                    side[key][3] = blueCorner[3]
                    blueFlag = True
                    break
                elif int(blueCorner[1]) == int(side[key][1]) and int(blueCorner[2])-1 == int(side[key][2]) and int(blueCorner[3]) == int(blueCorner[3]):
                    #side confirmed - update record
                    side[key][1] = blueCorner[1]
                    side[key][2] = blueCorner[2]
                    side[key][3] = blueCorner[3]
                    blueFlag=True
                    break
                elif int(blueCorner[1]) == int(side[key][1]) and int(blueCorner[2]) == int(side[key][2]) and int(blueCorner[3])-1 == int(blueCorner[3]):
                    #side confirmed - update record
                    side[key][1] = blueCorner[1]
                    side[key][2] = blueCorner[2]
                    side[key][3] = blueCorner[3]
                    blueFlag = True
                    break
            if fuzz.ratio(blueCornerFuzzy, str(value).lower().replace(" ", "").replace("-", "").replace("-", "").replace('"', '').replace("'", "")) >=90:
                if int(blueCorner[1])-1 == int(side[key][1]) and int(blueCorner[2]) == int(side[key][2]) and int(blueCorner[3]) == int(blueCorner[3]):
                    #side confirmed - add alias
                    side[key].append(blueCorner[0])
                    blueFlag = True
                    break
                elif int(blueCorner[1]) == int(side[key][1]) and int(blueCorner[2])-1 == int(side[key][2]) and int(blueCorner[3]) == int(blueCorner[3]):
                    #side confirmed - add alias
                    side[key].append(blueCorner[0])
                    blueFlag = True
                    break
                elif int(blueCorner[1]) == int(side[key][1]) and int(blueCorner[2]) == int(side[key][2]) and int(blueCorner[3])-1 == int(blueCorner[3]):
                    #side confirmed - add alias
                    side[key].append(blueCorner[0])
                    blueFlag = True
                    break
                
    if redFlag == False:
        side[uuid.uuid4()] = redCorner
    if blueFlag == False:
        side[uuid.uuid4()] = blueCorner

In [105]:
print(len(side))

4103
